# If the camera is moved:

1. run the "capture parking slots" cell
2. then run the "slot picker" cell, on the pop out window delete or add neccessary slots.
3. Then close the pop out window
4. Now, you can start the camera by running the "slot counter" cell.


note, if the camera is not disturbed/moved just run the slot counter cell


### capture parking slots

In [3]:
import requests
import time
import shutil

url='http://192.168.0.166'
r=requests.get(f'{url}/capture?_cb={int(round(time.time() * 1000))}', stream=True)

response=requests.get(f'{url}/capture?_cb={int(round(time.time() * 1000))}', stream=True)

with open('input/parking.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

ConnectTimeout: HTTPConnectionPool(host='192.168.0.166', port=80): Max retries exceeded with url: /capture?_cb=1664121916328 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001B3EE746590>, 'Connection to 192.168.0.166 timed out. (connect timeout=None)'))

 ### Slot picker

In [16]:
import cv2
import pickle
from math import sqrt

width, height = 160, 200
pt1_x, pt1_y, pt2_x, pt2_y = None, None, None, None
line_count = 0

try:
    with open('park_positions', 'rb') as f:
        park_positions = pickle.load(f)
except:
    park_positions = []


def parking_line_counter():
    global line_count
    line_count = int((sqrt((pt2_x - pt1_x) ** 2 + (pt2_y - pt1_y) ** 2)) / height)
    return line_count


def mouse_events(event, x, y, flag, param):
    global pt1_x, pt1_y, pt2_x, pt2_y

    if event == cv2.EVENT_LBUTTONDOWN:
        pt1_x, pt1_y = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        pt2_x, pt2_y = x, y
        parking_spaces = parking_line_counter()
        if parking_spaces == 0:
            park_positions.append((x, y))
        else:
            for i in range(parking_spaces):
                park_positions.append((pt1_x, pt1_y + i * height))

    if event == cv2.EVENT_RBUTTONDOWN:
        for i, position in enumerate(park_positions):
            x1, y1 = position
            if x1 < x < x1 + width and y1 < y < y1 + height:
                park_positions.pop(i)

    with open('park_positions', 'wb') as f:
        pickle.dump(park_positions, f)


while True:

    img = cv2.imread('input/parking.png')

    for position in park_positions:
        cv2.rectangle(img, position, (position[0] + width, position[1] + height), (255, 0, 255), 2)


    cv2.imshow('image', img)
    cv2.setMouseCallback('image', mouse_events)

    key = cv2.waitKey(30)
    if key == 27:
        break

cv2.destroyAllWindows()


### Slot counter

In [1]:
import cv2
import pickle
import urllib.request
import numpy as np

url='http://192.168.0.166/capture?'


with open('park_positions', 'rb') as f:
    park_positions = pickle.load(f)

font = cv2.FONT_HERSHEY_COMPLEX_SMALL

# Parking space parameters
width, height = 160, 200
full = width * height
empty = 150
parked_wrong = 200


def parking_space_counter(img_processed):
    global counter

    counter = 0

    for i,position in enumerate( park_positions):
        x, y = position

        img_crop = img_processed[y:y + height, x:x + width]
        count = cv2.countNonZero(img_crop)
        

        if count < empty:
            color = (0, 255, 0)
            counter += 1
            
        elif count<parked_wrong :
            color = (255, 0, 255)
           
        else:
            color = (0, 0, 255)

        cv2.rectangle(overlay, position, (position[0] + width, position[1] + height), color, -1)
        cv2.putText(overlay, "{:.2f}".format(count), (x + 4, y + height - 4), font, 0.7, (255, 255, 255), 1, cv2.LINE_AA)


while True:

 

    imgResp=urllib.request.urlopen(url)
    imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
    img=cv2.imdecode(imgNp,-1)

    overlay = img.copy()

    # Frame processing
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 1)
    img_thresh = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16)

    parking_space_counter(img_thresh)

    alpha = 0.7
    frame_new = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

    w, h = 50, 50
    cv2.rectangle(frame_new, (0, 0), (w, h), (255, 0, 255), -1)
    cv2.putText(frame_new, f"{counter}/{len(park_positions)}", (int(w / 10), int(h * 3 / 4)), font, 2, (255, 255, 255), 3, cv2.LINE_AA)


    cv2.imshow('frame', frame_new)
   

    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host